# Applied Data Analysis Course Project

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns= 163
pd.options.display.max_rows=163

## E. Build a database

You will build a database to hold your data. It is up to you to define appropriate tables and well as primary keys for connecting them. (The focus is definitely more on just using the basic methods/tools introduced in the course and not on building a complex database.) In particular, you can follow the following list of steps:

    restrict your data to 1000 entries and 5 columns of your choice
    create a connection to a sqlite3 database
    create one or multiple tables, at least one of the tables should have a PRIMARY KEY
    fill the database with your data
    run at least one query to demonstrate that it works correctly


In [2]:
df = pd.read_csv('/data/ml_course/02_Data_Analysis/en.openfoodfacts.org.products.csv', index_col=0, parse_dates=[4, 6])
df.head()

,code,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,brands,categories_en,origins,manufacturing_places,labels,emb_codes,first_packaging_code_geo,cities_tags,purchase_places,stores,countries_en,ingredients_text,allergens,allergens_en,traces_en,serving_size,additives_n,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states_en,main_category_en,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-montanic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
2,4559,usda-ndb-import,1489069957,2017-03-09 14:32:37+00:00,1489069957,2017-03-09 14:32:37+00:00,Peanuts,Unknown,Unknown,Unknown,Torn & Glasser,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,United States,"Peanuts, wheat flour, sugar, rice flour, tapio...",Unknown,Unknown,Unknown,28 g (0.25 cup),0.0,Unknown,0.0,Unknown,0.0,Unknown,b,Unknown,Unknown,"To be completed,Nutrition facts completed,Ingr...",Unknown,1941.0,NaN,17.86,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,60.71,17.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.1,17.86,NaN,NaN,NaN,0.63500,0.250,NaN,0.0,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.071,NaN,0.00129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,16087,usda-ndb-import,1489055731,2017-03-09 10:35:31+00:00,1489055731,2017-03-09 10:35:31+00:00,Organic Salted Nut Mix,Unknown,Unknown,Unknown,Grizzlies,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,United States,"Organic hazelnuts, organic cashews, organic wa...",Unknown,Unknown,Unknown,28 g (0.25 cup),0.0,Unknown,0.0,Unknown,0.0,Unknown,d,Unknown,Unknown,"To be completed,Nutrition facts completed,Ingr...",Unknown,2540.0,NaN,57.14,5.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.86,3.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.1,17.86,NaN,NaN,NaN,1.22428,0.482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.143,NaN,0.00514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0
4,16094,usda-ndb-import,1489055653,2017-03-09 10:34:13+00:00,1489055653,2017-03-09 10:34:13+00:00,Organic Polenta,Unknown,Unknown,Unknown,Bob's Red Mill,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,United States,Organic polenta,Unknown,Unknown,Unknown,35 g (0.25 cup),0.0,Unknown,0.0,Unknown,0.0,Unknown,Unknown,Unknown,Unknown,"To be completed,Nutritio

## Select 1000 entries

In [3]:
# Select only entries with completed data
df = df[df.states_en.str.contains('To be checked')]
df.shape

(60005, 122)

In [4]:
countries_en = df.countries_en.str.split(',', expand=True)
countries_en = countries_en.stack()
countries_en = countries_en.to_frame(name='countries_en')
countries_en['value'] = 1
countries_en = countries_en.groupby('countries_en').sum()
countries_en = countries_en.sort_values('value', ascending=False)
countries_en.head(50)

,value
countries_en,
France,47626
Germany,4754
Spain,3208
United Kingdom,1769
Switzerland,1631
Belgium,1056
United States,785
Australia,419
Portugal,340


In [5]:
# Remove data that have a duplicated product name
df = df[~df.product_name.duplicated(keep=False)]
df.shape

(45812, 122)

In [6]:
countries_en = df.countries_en.str.split(',', expand=True)
countries_en = countries_en.stack()
countries_en = countries_en.to_frame(name='countries_en')
countries_en['value'] = 1
countries_en = countries_en.groupby('countries_en').sum()
countries_en = countries_en.sort_values('value', ascending=False)
countries_en.head(50)

,value
countries_en,
France,35882
Germany,3623
Spain,2326
United Kingdom,1474
Switzerland,1227
Belgium,778
United States,667
Australia,361
Portugal,288


In [7]:
df.dropna(subset=['fat_100g', 'carbohydrates_100g', 'proteins_100g'], inplace=True)
df.shape

(43005, 122)

In [8]:
# Keep only data with a known country
df = df[~df.countries_en.str.contains('Unknown')]

In [9]:
countries_en = df.countries_en.str.split(',', expand=True)
countries_en = countries_en.stack()
countries_en = countries_en.to_frame(name='countries_en')
countries_en['value'] = 1
countries_en = countries_en.groupby('countries_en').sum()
countries_en = countries_en.sort_values('value', ascending=False)
countries_en.head(50)

,value
countries_en,
France,33745
Germany,3401
Spain,2253
United Kingdom,1278
Switzerland,1184
Belgium,708
United States,639
Australia,340
Portugal,275


In [10]:
# Remove duplicates of generic name
df = df[~df.generic_name.duplicated()]
df.shape

(25227, 122)

In [11]:
countries_en = df.countries_en.str.split(',', expand=True)
countries_en = countries_en.stack()
countries_en = countries_en.to_frame(name='countries_en')
countries_en['value'] = 1
countries_en = countries_en.groupby('countries_en').sum()
countries_en = countries_en.sort_values('value', ascending=False)
countries_en.head(50)

,value
countries_en,
France,18955
Germany,2533
Spain,1810
Switzerland,717
United Kingdom,610
Belgium,431
United States,248
Serbia,235
Portugal,230


In [12]:
# Drop entries with France since it is over represented
df.drop(df[df.countries_en == 'France'].sample(n=15000).index, inplace=True)
df.shape

(10227, 122)

In [13]:
# Remove duplicates of brands
df = df[~df.brands.duplicated()]
df.shape

(4767, 122)

In [14]:
countries_en = df.countries_en.str.split(',', expand=True)
countries_en = countries_en.stack()
countries_en = countries_en.to_frame(name='countries_en')
countries_en['value'] = 1
countries_en = countries_en.groupby('countries_en').sum()
countries_en = countries_en.sort_values('value', ascending=False)
countries_en.head(50)

,value
countries_en,
France,1732
Germany,1055
Spain,722
Switzerland,348
United Kingdom,272
Belgium,213
United States,209
Italy,141
Russia,127


In [15]:
df = df.sample(n=1000)
df.shape

(1000, 122)

## Select 5 columns

In [16]:
sel_col = ['product_name', 'countries_en', 'fat_100g', 'carbohydrates_100g', 'proteins_100g']
df = df.loc[:, sel_col]
df.head()

,product_name,countries_en,fat_100g,carbohydrates_100g,proteins_100g
310942,AKVO multivitamin aroma,Serbia,0.1,5.4,0.1
282610,Hanuta,"France,Germany",31.9,54.0,7.6
339831,Aceitunas verdes deshuesadas &quot;Jolca&quot;...,Spain,17.0,0.0,1.0
217411,Petit Camembert au Lait Cru,Germany,23.0,0.5,21.0
346905,Allinson volkoren,Netherlands,1.0,37.0,9.0


## Build database

In [17]:
db = sqlite3.connect("world_food.db")
cursor = db.cursor()

In [18]:
query= "CREATE TABLE world_food (product_name TEXT PRIMARY KEY, countries_en TEXT, fat_100g FLOAT, carbohydrates_100g FLOAT, proteins_100g FLOAT);"
cursor.execute(query)

In [19]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
results = cursor.fetchall()
print(results)

[('world_food',)]


In [20]:
df.to_sql(name='world_food', con=db, if_exists='append', index=False)

## Run query

In [21]:
def run_query(query):
    return pd.read_sql_query(query,db)

In [22]:
query = "SELECT * FROM world_food WHERE countries_en='Germany' LIMIT 20"
run_query(query)

,product_name,countries_en,fat_100g,carbohydrates_100g,proteins_100g
0,Petit Camembert au Lait Cru,Germany,23.0,0.5,21.0
1,Cocofin aus Kokosöl,Germany,100.0,0.0,0.0
2,Küstenbrause,Germany,0.0,10.0,0.0
3,Bourbon Vanille,Germany,7.4,28.0,2.5
4,Vollkornspaghetti,Germany,2.0,63.0,14.0
5,Rinder Hackfleisch,Germany,20.0,0.0,19.0
6,Huober Sticks,Germany,5.3,69.0,12.0
7,Schokoladen-Torte,Germany,25.0,40.0,9.5
8,Pfeffersalami im Pfeffermaltel geräuchert,Germany,26.0,1.0,21.0
9,merci Frohe Ostern,Germany,38.6,48.9,7.3
